# 6 Weighting & Aggregation

> *“Weight assignment is one of the most important steps in composite indicator*

I now have **21 normalised indicators** (after dropping the nine redundant ones).  Following the guidance in the handbook I will test **two transparent weighting schemes**:

| scheme | logic | compensability | why I include it |
|--------|-------|----------------|------------------|
| **Equal-by-pillar** | Each of the five pillars gets 20 %.<br>Inside a pillar the share is split equally. | fully compensatory | baseline that respects the theoretical balance of sub indices |
| **PCA-variance** | Weight<sub>q</sub> ∝ Σ<sub>j</sub> λ<sub>j</sub> · loading<sub>qj</sub><sup>2</sup>. | data-driven, still linear | rewards indicators that explain more common variance |


## 6.1 Equal weights by group

The equal-indicator approach (1 ÷ 21 ≈ 0.0476) *over-weights* sub indicators with many indicators. I instead give **each group 20 %**, then divide that share by the number of indicators
inside the group:

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

ROOT   = Path("..")
PROC   = ROOT / "data" / "processed"
ART    = ROOT / "artifacts" / "weights"
ART.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(PROC / "csiai_input_normalized.parquet")
if "ticker" in df.columns:
    df = df.set_index("ticker")

GROUPS = {
    "financial_strength": ["roe","debt_to_equity","current_ratio",
                           "oper_cash_flow","ebitda_margin"],
    "growth_potential":   ["revenue_growth","operating_margin","gross_margin"],
    "market_performance": ["eps","market_cap","price_to_sales","payout_ratio"],
    "risk_volatility":    ["hist_volatility","beta","max_drawdown",
                           "stddev_returns","value_at_risk"],
    "liquidity_trading":  ["avg_volume_30d","bid_ask_spread","volume_growth",
                           "float_shares"],
}

# equal-by-group weights
w_equal = {}
for group, cols in GROUPS.items():
    share = 0.20 / len(cols)
    print(f"Group: {group}, Share: {share}")
    w_equal.update({c: share for c in cols})

w_equal = pd.Series(w_equal, name="w_equal_group")
w_equal.to_csv(ART / "weights_equal_group.csv")

Group: financial_strength, Share: 0.04
Group: growth_potential, Share: 0.06666666666666667
Group: market_performance, Share: 0.05
Group: risk_volatility, Share: 0.04
Group: liquidity_trading, Share: 0.05
